In [1]:
import ibis

ibis.interactive = True

from pathlib import Path

import ibis.interactive
from streetscapes.streetview import SVWorkspace
from streetscapes.streetview.sources import GlobalStreetscapesSource
from streetscapes.streetview.sources import MapillarySource
from streetscapes.streetview.sources import KartaviewSource

In [2]:
ws = SVWorkspace("Amsterdam")

In [3]:
gss = GlobalStreetscapesSource(ws)

In [4]:
mp = MapillarySource(ws)

In [5]:
kv = KartaviewSource(ws)

In [6]:
kv.get_image_url(1208524)

'https://storage2.openstreetcam.org/files/photo/2016/5/5/proc/3823_3446d_572b6b13bdfca.jpg'

In [7]:
img_url = mp.get_image_url("4911230068985425")

In [8]:
img_url

'https://scontent-ams2-1.xx.fbcdn.net/m1/v/t6/An--rrF4WlM05JH5SMiWxbdSXVWL0mFEymquJWW0EmSjaBemmKBcF8DRGqIe8akNayRiH0VTYunIfeHZlt2zAGA0sYkEkeWIUtnfp27cgiCen37fra1_v0gwTJFgNL5wTiUGkS7F56f_yLBZF1HqZg?stp=s1024x768&_nc_oc=AdleHveUXlIpJYhf7Qx7TJdVir-WHEnQQv8pnkUUkh-zAJ8IXm8zrZvryHl7vCUrPUU&ccb=10-5&oh=00_AYFeoUG19L9sKicd9OjMnSpNXdZ8Ry24S9ZP_QaIiUs5ZQ&oe=68130D7C&_nc_sid=201bca'

In [7]:
gss.source_id

'NUS-UAL/global-streetscapes'

In [4]:
ws

In [ ]:
ws.local_dir, ws.csv_dir, ws.parquet_dir, ws.image_dirs

In [ ]:
subset = "amsterdam"

criteria = {"city": "Amsterdam"}

columns = {'uuid', 'source', 'city'}

ams = ws.load_subset(subset, criteria=criteria, columns=columns)

In [ ]:
ams.head()

In [ ]:
ams_loaded = ws.load_parquet_file("amsterdam")

In [ ]:
ams_loaded.head().select("uuid", "lon", "lat")

In [ ]:
ghsl = ws.load_global_parquet_file("streetscapes")

In [ ]:
ghsl.head()

In [ ]:
test_file = ws.get_file_path("test/test_file.txt")

In [ ]:
from huggingface_hub import hf_hub_download
from huggingface_hub import CachedRepoInfo
from huggingface_hub import try_to_load_from_cache
from huggingface_hub import cached_assets_path

In [ ]:
kwargs = {
    "repo_id": "NUS-UAL/global-streetscapes",
    "repo_type": "dataset",
}


# Update the file name.
#
# NOTE:
# HuggingFace replicates the structure of
# the remote repository, so any subdirectories
# should be included in the file name, not in
# the `local_dir` variable.
kwargs["filename"] = "data/ghsl.csv"

# Download the file
hf_hub_download(**kwargs)

In [ ]:
x = try_to_load_from_cache("NUS-UAL/global-streetscapes", "data/ghsl.csv", repo_type="dataset")

x

In [ ]:
cached_assets_path('streetscapes', namespace="images", subfolder="mapillary")

In [ ]:
ci = scan_cache_dir()

In [ ]:
for repo in ci.repos:
    if repo.repo_id == 'NUS-UAL/global-streetscapes':
        print(f"==[ repo: {repo.repo_path}")

In [ ]:
from pathlib import Path

conf = None

if conf is None and (local_env := Path.cwd() / ".env").exists():
    conf = local_env

conf